In [13]:
from Pegasus.api import *
from pathlib import Path

In [14]:
rc =  ReplicaCatalog()

with open("msg.txt", "w") as f:
    f.write(" Workflow 4 is almost done.")

msg = File("msg.txt")
rc.add_replica("local", msg, Path(".").resolve() / "msg.txt")
rc.write()

In [15]:
tc = TransformationCatalog()

t_container = Container(
            't_container',
            Container.DOCKER,
            image = "docker:///slnagark/workflow4:latest",
)

thumbsup = Transformation(
            "thumbsup",
            site="condorpool",
            pfn="/usr/bin/add_thumbs_up.py",
            is_stageable=False,
            container=t_container
)
tc.add_containers(t_container)
tc.add_transformations(thumbsup)
tc.write()

In [16]:
wf = Workflow("process")

result = File("result.txt")

job_thumbsup = Job(thumbsup)\
                .add_args('--i',msg, '--o',result)\
                .add_inputs(msg)\
                .add_outputs(result)
wf.add_jobs(job_thumbsup)

In [17]:
try:
    wf.plan(submit=True)\
        .wait()
except PegasusClientError as e:
    print(e)

2021.02.05 01:07:21.128 UTC:
2021.02.05 01:07:21.133 UTC:   -----------------------------------------------------------------------
2021.02.05 01:07:21.139 UTC:   File for submitting this DAG to HTCondor           : process-0.dag.condor.sub
2021.02.05 01:07:21.149 UTC:   Log of DAGMan debugging messages                 : process-0.dag.dagman.out
2021.02.05 01:07:21.154 UTC:   Log of HTCondor library output                     : process-0.dag.lib.out
2021.02.05 01:07:21.160 UTC:   Log of HTCondor library error messages             : process-0.dag.lib.err
2021.02.05 01:07:21.165 UTC:   Log of the life of condor_dagman itself          : process-0.dag.dagman.log
2021.02.05 01:07:21.170 UTC:
2021.02.05 01:07:21.176 UTC:   -no_submit given, not submitting DAG to HTCondor.  You can do this with:
2021.02.05 01:07:21.186 UTC:   -----------------------------------------------------------------------
2021.02.05 01:07:22.275 UTC:   Your database is compatible with Pegasus version: 5.1.0dev
2021.02

[####################################] 100.0% ..Success (Completed: 8, Queued: 0, Running: 0, Failed: 0)
